## NetCDF: working with fieldlists

In [1]:
import earthkit.data
earthkit.data.download_example_file("tuv_pl.nc")

In [2]:
ds = earthkit.data.from_source("file", "tuv_pl.nc")

### Iteration

We can itearte through the fields (we use the first 3 fields for simplicity):

In [3]:
for f in ds[:3]:
    print(f)

NetCDFField(t,level=1000,time=2018-08-01 12:00:00)
NetCDFField(t,level=850,time=2018-08-01 12:00:00)
NetCDFField(t,level=700,time=2018-08-01 12:00:00)


### Inspecting the contents

In [4]:
len(ds)

18

In [5]:
ds.ls()

,variable,level,valid_datetime,units
0,t,1000,2018-08-01T12:00:00,K
1,t,850,2018-08-01T12:00:00,K
2,t,700,2018-08-01T12:00:00,K
3,t,500,2018-08-01T12:00:00,K
4,t,400,2018-08-01T12:00:00,K
5,t,300,2018-08-01T12:00:00,K
6,u,1000,2018-08-01T12:00:00,m s**-1
7,u,850,2018-08-01T12:00:00,m s**-1
8,u,700,2018-08-01T12:00:00,m s**-1
9,u,500,2018-08-01T12:00:00,m s**-1


### Slicing

Standard Python slicing is available.

In [6]:
g = ds[1]
g

NetCDFField(t,level=850,time=2018-08-01 12:00:00)

In [7]:
g = ds[1:3]
g.ls()

,variable,level,valid_datetime,units
0,t,850,2018-08-01T12:00:00,K
1,t,700,2018-08-01T12:00:00,K


In [8]:
g = ds[-1]
g

NetCDFField(v,level=300,time=2018-08-01 12:00:00)

### Getting data values

#### Using values

In [9]:
v = ds[0].values
v.shape

(84,)

In [10]:
v[0:4]

array([272.56486405, 272.56486405, 272.56486405, 272.56486405])

In [11]:
v = ds.values
v.shape

(18, 84)

#### Using to_numpy()

In [12]:
v = ds[0].to_numpy()
print(v.shape)
print(ds[0].shape)

(7, 12)
(7, 12)


In [13]:
v = ds.to_numpy()
v.shape

(18, 7, 12)

### Metadata

Metadata access works both on individual fields and slices:

In [14]:
ds[0].metadata("variable")

't'

In [15]:
ds[0:2].metadata(["level", "variable"])

[[1000, 't'], [850, 't']]

and on all the fields:

In [16]:
ds.metadata("level")

[1000,
 850,
 700,
 500,
 400,
 300,
 1000,
 850,
 700,
 500,
 400,
 300,
 1000,
 850,
 700,
 500,
 400,
 300]

For each filed we can get the metadata as an object:

In [17]:
md = ds[0].metadata()
md

NetCDFMetadata({'units': 'K', 'long_name': 'Temperature', 'standard_name': 'air_temperature', 'date': 20180801, 'time': 1200, 'variable': 't', 'level': 1000, 'levtype': 'level'})

In [18]:
md["level"]

1000

### Selection

Selection by metadata is always creating a "view", no copying of data is involved.

In [19]:
g = ds.sel(variable=["u", "v"], level=850)
g.ls()

,variable,level,valid_datetime,units
0,u,850,2018-08-01T12:00:00,m s**-1
1,v,850,2018-08-01T12:00:00,m s**-1


In [20]:
g = ds.sel(variable="t")
g.ls()

,variable,level,valid_datetime,units
0,t,1000,2018-08-01T12:00:00,K
1,t,850,2018-08-01T12:00:00,K
2,t,700,2018-08-01T12:00:00,K
3,t,500,2018-08-01T12:00:00,K
4,t,400,2018-08-01T12:00:00,K
5,t,300,2018-08-01T12:00:00,K


### Xarray

Xarray conversion does not involve disk writing.

In [21]:
ds1 = ds.to_xarray()
ds1

<xarray.Dataset> Size: 12kB
Dimensions:    (longitude: 12, latitude: 7, level: 6, time: 1)
Coordinates:
  * longitude  (longitude) float32 48B 0.0 30.0 60.0 90.0 ... 270.0 300.0 330.0
  * latitude   (latitude) float32 28B 90.0 60.0 30.0 0.0 -30.0 -60.0 -90.0
  * level      (level) int32 24B 1000 850 700 500 400 300
  * time       (time) datetime64[ns] 8B 2018-08-01T12:00:00
Data variables:
    t          (time, level, latitude, longitude) float64 4kB dask.array<chunksize=(1, 6, 7, 12), meta=np.ndarray>
    u          (time, level, latitude, longitude) float64 4kB dask.array<chunksize=(1, 6, 7, 12), meta=np.ndarray>
    v          (time, level, latitude, longitude) float64 4kB dask.array<chunksize=(1, 6, 7, 12), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-08-07 18:24:35 GMT by grib_to_netcdf-2.30.2: grib_to_n...